# Predicting anomalies on NASA data using LSTM Pipeline

This is a demo notebook that shows how to use a pipeline with LSTM Regressor and
dynamic error thresholding primitives to detect anomalies in the NASA data.

In [1]:
import pandas as pd
from mlblocks import add_primitives_path, MLPipeline

In [2]:
from orion.data import load_signal

In [3]:
signal = 'S-1'

In [4]:
data = load_signal(signal)

In [5]:
data.shape

(10149, 2)

In [6]:
pipeline = MLPipeline.load('../orion/pipelines/lstm_dynamic_threshold.json')

Using TensorFlow backend.


In [7]:
pipeline.fit(data)

Epoch 1/3
9899/9899 [==============================] - 59s 6ms/step - loss: 0.0551 - mean_squared_error: 0.0551
Epoch 2/3
9899/9899 [==============================] - 59s 6ms/step - loss: 0.0525 - mean_squared_error: 0.0525
Epoch 3/3
9899/9899 [==============================] - 60s 6ms/step - loss: 0.0521 - mean_squared_error: 0.0521


In [9]:
anomalies = pipeline.predict(data)

/home/xals/.virtualenvs/Orion/lib/python3.6/site-packages/scipy/optimize/optimize.py:570: RuntimeWarning: invalid value encountered in subtract
  numpy.max(numpy.abs(fsim[0] - fsim[1:])) <= fatol):


In [10]:
adf = pd.DataFrame(anomalies, columns=['start', 'end', 'score'])
adf['start'] = adf['start'].astype(int)
adf['end'] = adf['end'].astype(int)
adf

,start,end,score
0,1222840800,1222862400,0.152513
1,1222884000,1222905600,0.121108
2,1397995200,1399507200,0.203102


In [12]:
pd.read_csv('labels.csv', index_col=0).loc[signal].values

array(['[[1392768000, 1402423200]]'], dtype=object)